## Example Sequence Alignment Viewer

See https://panel.pyviz.org/user_guide/index.html

In [2]:
import os, io, random
import string
import numpy as np
import pandas as pd
from importlib import reload

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

from IPython.display import HTML
import panel as pn
import panel.widgets as pnw

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, HoverTool, BoxSelectTool, Range1d, CustomJS, Slider
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot, column

from pybioviz import dashboards, utils, plotters

# pn.extension(raw_css=[utils.get_css()])

In [3]:
def make_seq():
    length =random.choice(range(45,50))
    return ''.join([random.choice(['A','C','T','G']) for i in range(length)])

In [4]:
fasta_file = os.path.join(utils.datadir, 'test.fa')

## test

In [5]:
reload(plotters)
seqs = ['TATAAACCGCATAACACGCAAGAACCG','CATACACGTCATAACACGCAAGAAA-A']
aln_file = 'tbd2_hits.aln'
aln = AlignIO.read(aln_file, 'clustal')

p = plotters.plot_sequence_alignment(aln,row_height=10)
pn.pane.Bokeh(p)

FileNotFoundError: [Errno 2] No such file or directory: 'tbd2_hits.aln'

## sequence aligner app

In [ ]:
reload(plotters)
reload(utils)

def sequence_alignment_viewer(filename=None):
    """Sequence alignment viewer"""
    
    title = pn.pane.Markdown()
    aln_btn = pnw.Button(name='align',width=100,button_type='primary')
    file_input = pnw.FileInput(name='load file',width=100,accept='.fa,.fasta,.faa')
    aligner_sel = pnw.Select(name='aligner',value='muscle',options=['muscle','clustal','maaft'],width=100)
    highlight_sel = pnw.Select(name='highlight mode',value='default',options=['default',''],width=100)
    rowheight_sl = pnw.IntSlider(name='row height',value=10,start=5,end=20,width=120)
    seq_pane = pn.pane.HTML(name='sequences',height=200,css_classes=['scrollingArea'])
    bokeh_pane = pn.pane.Bokeh()

    def update_title(filename):
        title.object = '### Sequence aligner: %s' %filename
        
    def update_file(event):        
        nonlocal seqtext
        seqtext = file_input.value.decode('utf-8')
        title.object = file_input.filename
        update_title(file_input.filename)
        return

    def align(event):
        #this function does the alignment
        nonlocal seqtext        
        if seqtext is not None:
            sequences = SeqIO.parse(io.StringIO(seqtext),format='fasta')
        elif filename is not None:    
            sequences = SeqIO.parse(filename, format='fasta')
        else:      
            return
        sequences = list(sequences)
        #print (sequences)
        aligner = aligner_sel.value
        if aligner == 'muscle':
            aln = utils.muscle_alignment(sequences)    
        elif aligner == 'clustal':
            aln = utils.clustal_alignment(sequences)
        elif aligner == 'mafft':
            aln = utils.mafft_alignment(sequences)
        if aln is None:
            bokeh_pane.object = plotters.plot_empty('%s not installed?' %aligner,900)
        else:
            #the bokeh pane is then updated with the new figure
            bokeh_pane.object = plotters.plot_sequence_alignment(aln, row_height=rowheight_sl.value)  
        return 
   
    seqtext = None
    file_input.param.watch(update_file,'value')
    rowheight_sl.param.watch(align,'value')
    aln_btn.param.watch(align, 'clicks')
    aln_btn.param.trigger('clicks')
    update_title(filename)
    side = pn.Column(aln_btn,file_input,aligner_sel,highlight_sel,rowheight_sl,seq_pane,css_classes=['form'],width=200,margin=20)   
    app = pn.Column(title,pn.Row(side, bokeh_pane), sizing_mode='stretch_width',width_policy='max',margin=20)
    return app

app = sequence_alignment_viewer('tbd2_hits.faa')
app